In [6]:
import numpy as np
import os
from scipy.spatial.transform import Rotation as R
from scipy.signal import butter, filtfilt, correlate, find_peaks
from scipy.interpolate import interp1d
from scipy.linalg import logm
from scipy.io import savemat, loadmat
import scipy.io as spio
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from vedo import Points, Plotter, Line, Axes, Plane, Grid
from IPython.display import Video
import imageio.v2 as imageio
import shutil
from pingouin import intraclass_corr

import matplotlib
import matplotlib.pyplot as plt

from mobgap.data import GenericMobilisedDataset
from mobgap.pipeline import MobilisedPipelineImpaired, MobilisedPipelineHealthy
from mobgap.utils.misc import get_env_var
from mobgap.aggregation import get_mobilised_dmo_thresholds

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [7]:
def get_day_boundaries(file_path):
    """Return (start, end) row indices for each day in the CSV."""
    time_col = pd.read_csv(file_path, usecols=["Time"])
    time_col["Time"] = pd.to_datetime(time_col["Time"], format="%Y-%m-%d %H:%M:%S.%f")

    days = time_col["Time"].dt.date
    change_idx = days.ne(days.shift()).to_numpy().nonzero()[0]
    change_idx = list(change_idx) + [len(days)]

    return [(change_idx[i], change_idx[i + 1]) for i in range(len(change_idx) - 1)]


def extract_day(file_path, start, end):
    """Read only rows between [start, end) from CSV."""
    nrows = end - start
    skip = (j for j in range(1, start + 1))  # generator → memory efficient
    df = pd.read_csv(
        file_path,
        skiprows=skip,
        nrows=nrows,
        parse_dates=["Time"],
        date_parser=lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"),
    )
    return df


def build_sensor_struct(df, fs, name):
    """Convert one day's dataframe into MATLAB struct for a sensor."""
    reformatted_time = (df["Time"].astype("int64") / 1e9).values.reshape(-1, 1)
    return {
        name: {
            "Fs": {"Acc": fs, "Gyr": fs},
            "Acc": df[["Accel-X (g)", " Accel-Y (g)", " Accel-Z (g)"]].values,
            "Gyr": df[[" Gyro-X (d/s)", " Gyro-Y (d/s)", " Gyro-Z (d/s)"]].values,
            "Timestamp": reformatted_time,
        }
    }


def load_metadata(raw_data_folder):
    """Read subject metadata once, return as infoForAlgo dict."""
    metadata = pd.read_excel(
        os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(raw_data_folder))))
        + r"\subject_metadata.xlsx"
    )
    subject_id = int(os.path.basename(os.path.dirname(os.path.dirname(raw_data_folder))))
    subject = metadata.loc[metadata["Subject_ID"] == subject_id]

    return {
        "TimeMeasure1": {
            "Subject_ID": subject["Subject_ID"].values[0],
            "Cohort": subject["Cohort"].values[0],
            "Gender": subject["Gender"].values[0],
            "Handedness": subject["Handedness"].values[0],
            "Age": subject["Age"].values[0],
            "Weight": subject["Weight"].values[0],
            "Height": subject["Height"].values[0],
            "SensorHeight": subject["SensorHeight"].values[0],
            "WalkingAid_01": subject["WalkingAid_01"].values[0],
            "WalkingAid_Side": subject["WalkingAid_Side"].values[0],
            "WalkingAid_Description": subject["WalkingAid_Description"].values[0],
            "SensorType_SU": subject["SensorType_SU"].values[0],
            "SensorAttachment_SU": subject["SensorAttachment_SU"].values[0],
        }
    }


def save_to_day_mats(raw_data_folder, regenerate):
    """
    Split wrist & waist CSVs into per-day data.mat files.
    Each file contains both sensors (if present), plus metadata.
    Skips days where data.mat already exists.
    """

    flag_file_path = os.path.join(raw_data_folder, "ALL_DAYS_DONE_FLAG")
    
    if regenerate:
        if os.path.exists(flag_file_path):
          os.remove(flag_file_path)
    
    if not os.path.isfile(flag_file_path):
        wrist_file = os.path.join(raw_data_folder, "wrist.resampled.csv")
        waist_file = os.path.join(raw_data_folder, "lower_back.resampled.csv")
    
        wrist_bounds = get_day_boundaries(wrist_file) if os.path.exists(wrist_file) else []
        waist_bounds = get_day_boundaries(waist_file) if os.path.exists(waist_file) else []
    
        n_days = max(len(wrist_bounds), len(waist_bounds))
        if prints:
            print(f"Detected {len(wrist_bounds)} wrist days, {len(waist_bounds)} waist days")
            print(f"Total output days: {n_days}")
    
        infoForAlgo = load_metadata(raw_data_folder)
    
        for day_idx in range(n_days):
            day_folder = os.path.join(raw_data_folder, f"Day {day_idx + 1}")
            data_path = os.path.join(day_folder, "data.mat")
    
            # Skip if already exists
            if os.path.exists(data_path) and not(regenerate):
                print(f"⏩ Skipping Day {day_idx+1}, already exists.")
                continue
    
            sensors = {}
    
            # Wrist for this day (if available)
            if day_idx < len(wrist_bounds):
                start, end = wrist_bounds[day_idx]
                df = extract_day(wrist_file, start, end)
                fs = float(round(len(df) / (df["Time"].iloc[-1] - df["Time"].iloc[0]).total_seconds()))
                sensors.update(build_sensor_struct(df, fs, "Wrist"))
    
            # Waist for this day (if available)
            if day_idx < len(waist_bounds):
                start, end = waist_bounds[day_idx]
                df = extract_day(waist_file, start, end)
                fs = float(round(len(df) / (df["Time"].iloc[-1] - df["Time"].iloc[0]).total_seconds()))
                sensors.update(build_sensor_struct(df, fs, "LowerBack"))
    
            # Skip empty days
            if not sensors:
                continue
    
            # Pick earliest timestamp across sensors for StartDateTime - should be midnight for both so not super important if there's a small difference
            all_times = [
                s["Timestamp"][0][0] for s in sensors.values() if len(s["Timestamp"]) > 0
            ]
            start_time = datetime.fromtimestamp(min(all_times)).strftime("%d-%b-%Y %H:%M:%S")
    
            data = {
                "TimeMeasure1": {
                    "Test1": {
                        "Trial1": {
                            "SU": sensors,
                            "StartDateTime": start_time,
                            "TimeZone": "Europe/UK",
                        }
                    }
                }
            }
    
            os.makedirs(day_folder, exist_ok=True)
    
            # Save combined .mat files
            savemat(data_path, {"data": data})
            savemat(os.path.join(day_folder, "infoForAlgo.mat"), {"infoForAlgo": infoForAlgo})
    
            print(f"✅ Saved Day {day_idx+1} with sensors: {list(sensors.keys())}")
    
        # create blank file as flag that all days are done
        with open(flag_file_path, "w") as file:
            pass

In [8]:
prints = False
regenerate_data_files = False

base_path = r"C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset"

# Loop through all the participant folders syncing axivity sensors and mocap timings
for cohort in os.listdir(base_path):
    cohort_path = os.path.join(base_path, cohort)
    if os.path.isdir(cohort_path):
        for participant in os.listdir(cohort_path):
            participant_path = os.path.join(cohort_path, participant)
            if os.path.isdir(participant_path):
                participant_path = os.path.join(participant_path, 'Home\\')
                print(participant_path)
                save_to_day_mats(participant_path, regenerate = regenerate_data_files)

C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\383\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\582\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\718\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\855\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\CP\875\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\HA\3\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\PSP\864\Home\
C:\Users\ac4jmi\Desktop\DMO4LNC\Data Collection\Dataset\PSP\899\Home\


In [9]:
def get_paths_with_extension(extension, dataset_folder_name="Dataset"):
    paths = []
    dataset_folder = os.path.join(os.getcwd(), dataset_folder_name)

    for root, dirs, files in os.walk(dataset_folder):
        # Skip any folder path containing "\Lab\"
        if "Lab" in root.split(os.sep):
            continue

        for file in files:
            if file.lower().endswith(extension):
                path = os.path.join(root, file)
                paths.append(path)
    
    return paths

def get_mobilised_dataset():

    print("Building dataset from .mat files...")
    paths_list = get_paths_with_extension("data.mat")
    # Build a dataset from the .mat files
    dataset = GenericMobilisedDataset(
        paths_list = paths_list,
        test_level_names = ["TimeMeasure", "Test", "Trial"], # This is the structure (without numbers) of the data inside the data.mat struct.
        measurement_condition = "laboratory", # I'm not currently sure what this does on the algorithm side of things, can also be "free_living".
        parent_folders_as_metadata=["cohort", "subject_id", "location", "day"] # these are the column headings for the metadata, the columns get populated with the folder names.
    )

    return dataset

# get mobgap dataset
dataset = get_mobilised_dataset()
print(dataset)
subject_metadata = pd.read_excel("Dataset/subject_metadata.xlsx")

cohorts = ["CP", "PSP"]

all_pipeline_day_wb_params = pd.DataFrame()
all_pipeline_day_agg_params = pd.DataFrame()

# get the HA healthy thresholds and rename them to "CP" so that we can use them to do some basic thresholding on the data
ha_thresholds = get_mobilised_dmo_thresholds().xs("HA", level=1, drop_level=False)
new_index = pd.MultiIndex.from_tuples([(dmo, cohort) for dmo, _ in ha_thresholds.index for cohort in cohorts], names=ha_thresholds.index.names)
duplicated_values = pd.concat([ha_thresholds] * len(cohorts), axis=0).reset_index(drop=True)
multi_cohort_thresholds = pd.DataFrame(duplicated_values.values, index=new_index, columns=ha_thresholds.columns)

for cohort in cohorts:
    data = dataset.get_subset(cohort=cohort)
    subjects = list({row[1] for row in dataset.group_labels if row[0] == cohort})
    
    for subject in subjects:
        for day in [f"Day {x}" for x in range(1, 11)]:
            try:
                test = data.get_subset(Test="Test1", day=day, subject_id=str(subject))
            except KeyError as k:
                msg = str(k)
                if "No datapoint in the dataset matched the following filter" in msg:
                    print(f"{day} does not exist for subject {subject}")
                    continue
                # raise unexpected key errors
                raise
                
            try:
                pipeline = MobilisedPipelineImpaired(dmo_thresholds=multi_cohort_thresholds)
                pipeline = pipeline.safe_run(test)
                meta = subject_metadata.loc[subject_metadata["Subject_ID"] == int(subject)]
                
                day_wb_started = pd.to_timedelta(pipeline.per_wb_parameters_['start'], unit='ms') + test.data["LowerBack"].index[0]
                day_wb_started = day_wb_started.dt.day_name()
                print(day_wb_started)
                pipeline.per_wb_parameters_.insert(loc = 0, column = "subject_id", value = str(subject))
                pipeline.per_wb_parameters_.insert(loc = 1, column = "day", value = day)
                pipeline.per_wb_parameters_.insert(loc = 2, column = "day_name", value = day_wb_started)
                pipeline.per_wb_parameters_.insert(loc = 3, column = "cohort", value = cohort)
                pipeline.per_wb_parameters_.insert(loc = 4, column = "height", value = meta["Height"].values[0])
                pipeline.per_wb_parameters_.insert(loc = 5, column = "weight", value = meta["Weight"].values[0])
                pipeline.per_wb_parameters_.insert(loc = 6, column = "age", value = meta["Age"].values[0])
                pipeline.per_wb_parameters_.insert(loc = 7, column = "gender", value = meta["Gender"].values[0])
                pipeline.per_wb_parameters_.insert(loc = 8, column = "handedness", value = meta["Handedness"].values[0])
                # add time offset to wb parameters to get absolute timestamps
                pipeline.per_wb_parameters_['start'] = pd.to_timedelta(pipeline.per_wb_parameters_['start']*10, unit='ms') + test.data["LowerBack"].index[0]
                pipeline.per_wb_parameters_['end'] = pd.to_timedelta(pipeline.per_wb_parameters_['end']*10, unit='ms') + test.data["LowerBack"].index[0]

                
                pipeline.aggregated_parameters_.insert(loc = 0, column = "subject_id", value = str(subject))
                pipeline.aggregated_parameters_.insert(loc = 1, column = "day", value = day)
                pipeline.aggregated_parameters_.insert(loc = 2, column = "day_name", value = day_wb_started.values[0])
                pipeline.aggregated_parameters_.insert(loc = 3, column = "cohort", value = cohort)
                pipeline.aggregated_parameters_.insert(loc = 4, column = "height", value = meta["Height"].values[0])
                pipeline.aggregated_parameters_.insert(loc = 5, column = "weight", value = meta["Weight"].values[0])
                pipeline.aggregated_parameters_.insert(loc = 6, column = "age", value = meta["Age"].values[0])
                pipeline.aggregated_parameters_.insert(loc = 7, column = "gender", value = meta["Gender"].values[0])
                pipeline.aggregated_parameters_.insert(loc = 8, column = "handedness", value = meta["Handedness"].values[0])
                
                all_pipeline_day_wb_params = pd.concat([all_pipeline_day_wb_params, pipeline.per_wb_parameters_]).reset_index(drop=True)
                all_pipeline_day_agg_params = pd.concat([all_pipeline_day_agg_params, pipeline.aggregated_parameters_]).reset_index(drop=True)
                
            except ValueError as e:
                msg = str(e)
                if "Expected sensor data for" in msg and "LowerBack" in msg:
                    print(f"Skipping {day} — missing LowerBack sensor")
                    continue
                # raise unexpected value errors
                raise

# drop columns we don't want
all_pipeline_day_wb_params = all_pipeline_day_wb_params.drop(["rule_name", "rule_obj"], axis=1)

all_pipeline_day_wb_params.to_csv(r"Free-Living Parameters/per_wb_parameters.csv")
all_pipeline_day_agg_params.to_csv(r"Free-Living Parameters/all_day_parameters.csv")

display(all_pipeline_day_wb_params)
display(all_pipeline_day_agg_params)

fig, axes = plt.subplots(2, 3, figsize=(12, 5))
axes = axes.flatten()
# histogram of durations
axes[0].hist(all_pipeline_day_wb_params["duration_s"], bins=100, edgecolor="black")
axes[0].set_title("Walking Bout Duration")
axes[0].set_xlabel("Duration (s)")
axes[0].set_ylabel("Count")

# histogram of stride counts
axes[1].hist(all_pipeline_day_wb_params["n_strides"], bins=100, edgecolor="black")
axes[1].set_title("Number of Strides per Walking Bout")
axes[1].set_xlabel("Number of Strides per Walking Bout")
axes[1].set_ylabel("Count")

# histogram of stride counts
axes[2].hist(all_pipeline_day_wb_params["stride_duration_s"], bins=100, edgecolor="black")
axes[2].set_title("Stride Duration per Walking Bout")
axes[2].set_xlabel("Stride Duration (s)")
axes[2].set_ylabel("Count")

# histogram of stride counts
axes[3].hist(all_pipeline_day_wb_params["cadence_spm"], bins=100, edgecolor="black")
axes[3].set_title("Cadence per Walking Bout")
axes[3].set_xlabel("Cadence (steps/min)")
axes[3].set_ylabel("Count")

# histogram of stride counts
axes[4].hist(all_pipeline_day_wb_params["stride_length_m"], bins=100, edgecolor="black")
axes[4].set_title("Stride Length per Walking Bout")
axes[4].set_xlabel("Stride Length (m)")
axes[4].set_ylabel("Count")

# histogram of stride counts
axes[5].hist(all_pipeline_day_wb_params["walking_speed_mps"], bins=100, edgecolor="black")
axes[5].set_title("Walking Speed per Walking Bout")
axes[5].set_xlabel("Walking Speed (mps)")
axes[5].set_ylabel("Count")

plt.tight_layout()
plt.show()

Building dataset from .mat files...
GenericMobilisedDataset [66 groups/rows]

      cohort subject_id location    day   TimeMeasure   Test   Trial
   0      CP        383     Home  Day 1  TimeMeasure1  Test1  Trial1
   1      CP        383     Home  Day 2  TimeMeasure1  Test1  Trial1
   2      CP        383     Home  Day 3  TimeMeasure1  Test1  Trial1
   3      CP        383     Home  Day 4  TimeMeasure1  Test1  Trial1
   4      CP        383     Home  Day 5  TimeMeasure1  Test1  Trial1
   ..    ...        ...      ...    ...           ...    ...     ...
   61    PSP        899     Home  Day 5  TimeMeasure1  Test1  Trial1
   62    PSP        899     Home  Day 6  TimeMeasure1  Test1  Trial1
   63    PSP        899     Home  Day 7  TimeMeasure1  Test1  Trial1
   64    PSP        899     Home  Day 8  TimeMeasure1  Test1  Trial1
   65    PSP        899     Home  Day 9  TimeMeasure1  Test1  Trial1
   
   [66 rows x 7 columns]
wb_id
0      Wednesday
1      Wednesday
2      Wednesday
3      W

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# --- Existing aggregation ---
df = all_pipeline_day_wb_params.copy(deep=True)
df['start'] = pd.to_datetime(df['start'])
df['hour_temp'] = df['start'].dt.floor('H')
df['hour'] = df['hour_temp'].dt.strftime('%H:%M:%S')

sum_cols = ['n_strides', 'duration_s', 'n_raw_initial_contacts']
mean_cols = ['duration_s', 'stride_duration_s', 'cadence_spm', 'stride_length_m', 'walking_speed_mps']

agg_dict = {}
for col in sum_cols:
    agg_dict[f"{col}_sum"] = (col, 'sum')
for col in mean_cols:
    agg_dict[f"{col}_mean"] = (col, 'mean')

grouped = (
    df.groupby(['subject_id', 'day', 'day_name', 'cohort', 'hour'], as_index=False)
      .agg(**agg_dict)
)

id_cols = ['subject_id', 'day', 'day_name', 'cohort', 'hour']
metric_cols = [c for c in grouped.columns if c not in id_cols]
grouped = grouped[id_cols + metric_cols]


# --- STEP 1: Create a full 24-hour template for each subject/day ---
hours_full = pd.date_range('00:00:00', '23:00:00', freq='H').strftime('%H:%M:%S')

subjects_days = grouped[['subject_id', 'day', 'day_name', 'cohort']].drop_duplicates()

template = (
    subjects_days
    .assign(key=1)
    .merge(pd.DataFrame({'hour': hours_full, 'key': 1}), on='key')
    .drop('key', axis=1)
)

# --- STEP 2: Merge your grouped data with the full template ---
merged = pd.merge(
    template,
    grouped,
    on=['subject_id', 'day', 'day_name', 'cohort', 'hour'],
    how='left'
)

merged = merged.astype({
    'n_strides_sum': 'float64',
    'duration_s_sum': 'float64',
    'n_raw_initial_contacts_sum': 'float64'
})

# --- STEP 3: Optional reorder and display ---
cols = ['subject_id', 'day', 'day_name', 'cohort', 'hour'] + metric_cols
merged = merged[cols]

display(merged.head(10))

merged.to_csv(r"Free-Living Parameters/hourly_grouped_parameters.csv")

In [ ]:
# to do:
# - create tools to plot graphs based on weight, height, age, gender, etc.